# Manipulating Data in DataFrames HW


#### Let's get started applying what we learned in the lecure!

I've provided several questions below to help test and expand you knowledge from the code along lecture. So let's see what you've got!

First create your spark instance as we need to do at the start of every project.

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName(f"SparkApp").getOrCreate()


## Read in our Republican vs. Democrats Tweet DataFrame

Attached to the lecture

In [2]:
dataset_path = 'dataset\ExtractedTweets.csv'

#read data
data = spark.read.csv(dataset_path, header=True)

## About this dataframe

Extracted tweets from all of the representatives (latest 200 as of May 17th 2018)

**Source:** https://www.kaggle.com/kapastor/democratvsrepublicantweets#ExtractedTweets.csv

Use either .show() or .toPandas() check out the first view rows of the dataframe to get an idea of what we are working with.

In [3]:
data.show(5)

data.limit(5).toPandas()

+--------------------+-------------+--------------------+
|               Party|       Handle|               Tweet|
+--------------------+-------------+--------------------+
|            Democrat|RepDarrenSoto|Today, Senate Dem...|
|            Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|            Democrat|RepDarrenSoto|RT @NBCLatino: .@...|
|Congress has allo...|         NULL|                NULL|
|            Democrat|RepDarrenSoto|RT @NALCABPolicy:...|
+--------------------+-------------+--------------------+
only showing top 5 rows



,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,"Congress has allocated about $18…""",None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...


**Prevent Truncation of view**

If the view you produced above truncated some of the longer tweets, see if you can prevent that so you can read the whole tweet.

In [4]:
data.show(truncate=False)

+----------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party                                               |Handle       |Tweet                                                                                                                                       |
+----------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat                                            |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|Democrat                                            |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized 

**Print Schema**

First, check the schema to make sure the datatypes are accurate. 

In [5]:
data.printSchema()

root
 |-- Party: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- Tweet: string (nullable = true)



## 1. Can you identify any tweet that mentions the handle @LatinoLeader using regexp_extract?

It doesn't matter how you identify the row, any identifier will do. You can test your script on row 5 from this dataset. That row contains @LatinoLeader. 

In [6]:
from pyspark.sql.functions import regexp_extract, col

pattern = '(.)(@LatinoLeader)(.)'
df = data.withColumn('Latino_mentions', regexp_extract(col('Tweet'), pattern,2))
df.show(truncate=False)

+----------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|Party                                               |Handle       |Tweet                                                                                                                                       |Latino_mentions|
+----------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|Democrat                                            |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |               |
|Democrat                                            |RepDarrenSoto|RT @WinterHavenSun: Winter H

## 2. Replace any value other than 'Democrate' or 'Republican' with 'Other' in the Party column.

We can see from the output below, that there are several other values other than 'Democrate' or 'Republican' in the Part column. We are assuming that this is dirty data that needs to be cleaned up.

In [7]:
from pyspark.sql.functions import desc
count = data.groupBy('Party').count()
#count Parties
count.orderBy(desc('count')).show()



+--------------------+-----+
|               Party|count|
+--------------------+-----+
|          Republican|44392|
|            Democrat|42068|
|            That’s…"|   28|
|https://t.co/oc6J...|   22|
|                 Now|   17|
|               Today|   13|
|         https://t…"|   12|
|              http…"|   12|
|                 h…"|   12|
|               ❌ E…"|   11|
|❌ Abandoned Hispa...|   11|
|   ❌ Passed #TaxScam|   11|
|  ❌ Terminated #DACA|   11|
|        #mepolitics"|   11|
|https://t.co/8htz...|    9|
|      https://t.co…"|    9|
|💻 Website: https...|    9|
|This. Has. To. St...|    8|
|                   4|    7|
|Semper Fidelis. h...|    7|
+--------------------+-----+
only showing top 20 rows



## 3. Delete all embedded links (ie. "https:....)

For example see the first row in the tweets dataframe. 

*Note: this may require an google search :)*

In [8]:
from pyspark.sql.functions import regexp_replace
pattern = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'  # Regular expression pattern to match URLs
clean = data.withColumn('clean', regexp_replace(data.Tweet, pattern, '')).select('clean')
clean.show(truncate = False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|clean                                                                                                                                       |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House…                         |
|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.                                    |
|NULL                                                                                                                                        |

## 4. Remove any leading or trailing white space in the tweet column

In [9]:
from pyspark.sql.functions import trim

df = data.withColumn('Tweet', trim(data['Tweet']))
# Now, all the tweets have been clean
df.select('Tweet').show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|Tweet                                                                                                                                       |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.                                    |
|NULL                                                                                                                                        |

## 5. Rename the 'Party' column to 'Dem_Rep'

No real reason here :) just wanted you to get practice doing this. 

In [10]:
data_renamed = data.withColumnRenamed("Party", 'Dem_Rep')
data_renamed.printSchema()


root
 |-- Dem_Rep: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- Tweet: string (nullable = true)



## 6. Concatenate the Party and Handle columns

Silly yes... but good practice.

pyspark.sql.functions.concat_ws(sep, *cols)[source] <br>
Concatenates multiple input string columns together into a single string column, using the given separator.

In [11]:
from pyspark.sql.functions import concat_ws
data = data.select(concat_ws(" ", "Party", "Handle").alias("Party_Handle"))
data.show()

+--------------------+
|        Party_Handle|
+--------------------+
|Democrat RepDarre...|
|Democrat RepDarre...|
|Democrat RepDarre...|
|Congress has allo...|
|Democrat RepDarre...|
|Democrat RepDarre...|
|Democrat RepDarre...|
|Democrat RepDarre...|
|Democrat RepDarre...|
|Democrat RepDarre...|
|Democrat RepDarre...|
|Democrat RepDarre...|
|Democrat RepDarre...|
|Democrat RepDarre...|
|Democrat RepDarre...|
|Sgt Sam Howard - ...|
|Democrat RepDarre...|
|Democrat RepDarre...|
|Democrat RepDarre...|
|Democrat RepDarre...|
+--------------------+
only showing top 20 rows



## Challenge Question

Let's image that we want to analyze the hashtags that are used in these tweets. Can you extract all the hashtags you see?

# Let's create our own dataset to work with real dates

This is a dataset of patient visits from a medical office. It contains the patients first and last names, date of birth, and the dates of their first 3 visits. 

In [14]:
from pyspark.sql.types import *

md_office = [('Mohammed','Alfasy','1987-4-8','2016-1-7','2017-2-3','2018-3-2') \
            ,('Marcy','Wellmaker','1986-4-8','2015-1-7','2017-1-3','2018-1-2') \
            ,('Ginny','Ginger','1986-7-10','2014-8-7','2015-2-3','2016-3-2') \
            ,('Vijay','Doberson','1988-5-2','2016-1-7','2018-2-3','2018-3-2') \
            ,('Orhan','Gelicek','1987-5-11','2016-5-7','2017-1-3','2018-9-2') \
            ,('Sarah','Jones','1956-7-6','2016-4-7','2017-8-3','2018-10-2') \
            ,('John','Johnson','2017-10-12','2018-1-2','2018-10-3','2018-3-2') ]

df = spark.createDataFrame(md_office,['first_name','last_name','dob','visit1','visit2','visit3']) # schema=final_struc

# Check to make sure it worked
df.show()
print(df.printSchema())

+----------+---------+----------+--------+---------+---------+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|
+----------+---------+----------+--------+---------+---------+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|
+----------+---------+----------+--------+---------+---------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- visit1: string (nullable = true)
 |-- visit2: string (nullable = true)
 |-- visit3: string (nullable = true)

None


Oh no! The dates are still stored as text... let's try converting them again and see if we have any issues this time.

In [3]:
from pyspark.sql.functions import col, to_date, expr
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DateType
spark1 = SparkSession.builder.appName("Patients_df").getOrCreate()


md_office = [('Mohammed','Alfasy','1987-4-8','2016-1-7','2017-2-3','2018-3-2') \
            ,('Marcy','Wellmaker','1986-4-8','2015-1-7','2017-1-3','2018-1-2') \
            ,('Ginny','Ginger','1986-7-10','2014-8-7','2015-2-3','2016-3-2') \
            ,('Vijay','Doberson','1988-5-2','2016-1-7','2018-2-3','2018-3-2') \
            ,('Orhan','Gelicek','1987-5-11','2016-5-7','2017-1-3','2018-9-2') \
            ,('Sarah','Jones','1956-7-6','2016-4-7','2017-8-3','2018-10-2') \
            ,('John','Johnson','2017-10-12','2018-1-2','2018-10-3','2018-3-2') ]
final_struct = StructType([
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("dob", StringType(), True),
    StructField("visit1", StringType(), True),
    StructField("visit2", StringType(), True),
    StructField("visit3", StringType(), True)
])

df = spark1.createDataFrame(md_office, schema=final_struct)


df = df.withColumn("dob", to_date(expr("lpad(dob,10,'0')"), "yyyy-MM-dd")) \
       .withColumn("visit1", to_date(expr("lpad(visit1,10,'0')"), "yyyy-MM-dd")) \
       .withColumn("visit2", to_date(expr("lpad(visit2,10,'0')"), "yyyy-MM-dd")) \
       .withColumn("visit3", to_date(expr("lpad(visit3,10,'0')"), "yyyy-MM-dd"))
       
df.printSchema()
df.show()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- visit1: date (nullable = true)
 |-- visit2: date (nullable = true)
 |-- visit3: date (nullable = true)



Py4JJavaError: An error occurred while calling o84.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (DESKTOP-RTUQ6MF executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoin

## 7. Can you calculate a variable showing the length of time between patient visits?

Compare visit1 to visit2 and visit2 to visit3 for all patients and see what the average length of time is between visits. Create an alias for it as well. 

In [ ]:
from pyspark.sql.functions import datediff, col
duration1 = df.select(datediff(col("visit1"), col("visit2")).alias('visit1_visit_2'), datediff(col("visit2"), col("visit3")).alias('visit2_visit3'))

duration1.show()

Py4JJavaError: An error occurred while calling o368.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 39.0 failed 1 times, most recent failure: Lost task 0.0 in stage 39.0 (TID 82) (DESKTOP-RTUQ6MF executor driver): org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse '2017-2-3' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError(ExecutionErrors.scala:54)
	at org.apache.spark.sql.errors.ExecutionErrors.failToParseDateTimeInNewParserError$(ExecutionErrors.scala:48)
	at org.apache.spark.sql.errors.ExecutionErrors$.failToParseDateTimeInNewParserError(ExecutionErrors.scala:218)
	at org.apac

## 8. Can you calculate the age of each patient?

## 9. Can you extract the month from the first visit column and call it "Month"?

## 10. Challenges with working with date and timestamps

Let's read in the supermarket sales dataframe attached to the lecture now and see some of the issues that can come up when working with date and timestamps values.

## About this dataset

The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. 

 - Attribute information
 - Invoice id: Computer generated sales slip invoice identification number
 - Branch: Branch of supercenter (3 branches are available identified by A, B and C).
 - City: Location of supercenters
 - Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
 - Gender: Gender type of customer
 - Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
 - Unit price: Price of each product in USD
 - Quantity: Number of products purchased by customer
 - Tax: 5% tax fee for customer buying
 - Total: Total price including tax
 - Date: Date of purchase (Record available from January 2019 to March 2019)
 - Time: Purchase time (10am to 9pm)
 - Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
 - COGS: Cost of goods sold
 - Gross margin percentage: Gross margin percentage
 - Gross income: Gross income
 - Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

**Source:** https://www.kaggle.com/aungpyaeap/supermarket-sales

### View dataframe and schema as usual

### Convert date field to date type

Looks like we need to convert the date field into a date type. Let's go ahead and do that..

### How can we extract the month value from the date field?

If you had trouble converting the date field in the previous question think about a more creative solution to extract the month from that field.

## 11.0 Working with Arrays

Here is a dataframe of reviews from the movie the Dark Night.

In [ ]:
from pyspark.sql.functions import *

values = [(5,'Epic. This is the best movie I have EVER seen'), \
          (4,'Pretty good, but I would have liked to seen better special effects'), \
          (3,'So so. Casting could have been improved'), \
          (5,'The most EPIC movie of the year! Casting was awesome. Special effects were so intense.'), \
          (4,'Solid but I would have liked to see more of the love story'), \
          (5,'THE BOMB!!!!!!!')]
reviews = spark.createDataFrame(values,['rating', 'review_txt'])

reviews.show(6,False)

+------+--------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |
+------+--------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |
|4     |Pretty good, but I would have liked to seen better special effects                    |
|3     |So so. Casting could have been improved                                               |
|5     |The most EPIC movie of the year! Casting was awesome. Special effects were so intense.|
|4     |Solid but I would have liked to see more of the love story                            |
|5     |THE BOMB!!!!!!!                                                                       |
+------+--------------------------------------------------------------------------------------+



## 11.1 Let's see if we can create an array off of the review text column and then derive some meaningful results from it.

**But first** we need to clean the rview_txt column to make sure we can get what we need from our analysis later on. So let's do the following:

1. Remove all punctuation
2. lower case everything
3. Remove white space (trim)
3. Then finally, split the string

## 11.2 Alright now let's see if we can find which reviews contain the word 'Epic'

### That's it! Great Job!